## TAX Assistant Chatbot with RAG & GROQ Inferencing

##### Conda command to install the code with all the requirements
conda create -n rahul_taxbot python=3.10 -c conda-forge --file requirements.txt


In [1]:
# Loading the PDF Files
from langchain_community.document_loaders import PyPDFDirectoryLoader 
loader=PyPDFDirectoryLoader("../Tax PDF Data")
docs = loader.load()

In [2]:
# Transforming the documents in chunks 
from langchain.text_splitter import  RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='F. No.370142/6/2024- TPL \nGovernment of India \nMinistry of Finance \nDepartment of Revenue \nCentral Board of Direct Taxes \n(TPL Division) \nSub: Order under section 119 of the Income-tax Act, 1961 -reg. Circular No. ;2../2024 \nDated osth March, 2024 \nIncome of any fund or institution or trust or any university or other educational \ninstitution or any hospital or other medical institution referred to in sub-clause (iv) or sub\xad\nclause (v) or sub-clause (vi) or sub-clause (via) of clause (23C) of section 10 (hereinafter \nreferred to as trust or institution under the first regime) of the Income-tax Act, 1961 \n(hereinafter referred to as the Act) or any trust or institution registered under section 12AA or \nsection 12AB of the Act (hereinafter referred to as trust or institution under the second \nregime) is exempt subject to fulfilment of certain conditions specified under various sections \nof the Act.', metadata={'source': '..\\Tax PDF Data\\circular

In [3]:
# FAISS Vector Database
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
db = FAISS.from_documents(documents,OpenAIEmbeddings())
db

In [4]:
# Using Opensource LLM - Llama3-8b Model
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

llm=ChatGroq(groq_api_key = groq_api_key,model_name="Llama3-8b-8192")

In [5]:
# Prompt Template Creation
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Consider you are a smart tax assistant who has very high knowledge in the latest and accurate Indian tax. 
            I want you to answer my question {input} in a very compact and precise manner within 200 words
            Give all the answers with respect to Indian tax context only. 
            Return the output in a clean and crisp answer 
            <context>
            {context}
            </context>
            Question:{input}
            """)

retriver -> interface to vector store
context  -> documents data stored in vector store

In [6]:
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001349C16CBE0>)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt) 

In [8]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver,document_chain)


#### Few Examples on model performance against user input

In [9]:
retrieval_chain.invoke({"input": "do i need to store income proofs"})

{'input': 'do i need to store income proofs',
 'context': [Document(page_content='reasonable records with which you can support the claim of income.\nAs an agriculturist, am I required to maintain any proof of earnings and expenditures\nincurred?\n  Even if you have only agricultural income, you are advised to maintain some proof of your\nagricultural earnings/expenses.\nIf I win a lottery or prize money in a competition, am I required to pay Income-tax on it?\n  Yes, such  winnings  are liable to flat rate of tax at 30% without any basic exemption limit. In\nsuch a case the payer of prize money will generally deduct tax at source (i.e., TDS) from the\nwinnings and will pay you only the balance amount.', metadata={'source': '..\\Tax PDF Data\\Income Tax India Frequently Asked Questions.pdf', 'page': 2}),
  Document(page_content='might need this form.\n6. Proof of Tax Saving Investments: Keep documents supporting your investments\nin tax-saving instruments like LIC, PPF, NSC, etc.\n7. B

In [16]:
response = retrieval_chain.invoke({"input": "do i need to store income proofs"})
print(response['answer'])

Yes, you need to store income proofs. Under the Indian Income-tax Act, it is mandatory to maintain proof of earnings and records for all sources of income. This is essential to support your income claims during tax assessments or audits. Even if no specific records are prescribed, you should keep reasonable documentation. For agricultural income, although it is generally exempt from tax, maintaining records of earnings and expenditures is advisable. Ensure that you also keep proof of tax payments, such as the stamped counter-foil of the Income-tax Challan, and check your tax credit via Form 26AS from your e-filing account.


In [10]:
response = retrieval_chain.invoke({"input": "what are the tax rates under the new tax regime"})
print(response['answer'])

Under the new tax regime in India, the income tax rates are as follows:

- Up to ₹3,00,000: Nil
- ₹3,00,001 to ₹6,00,000: 5%
- ₹6,00,001 to ₹9,00,000: 10%
- ₹9,00,001 to ₹12,00,000: 15%
- ₹12,00,001 to ₹15,00,000: 20%
- Above ₹15,00,000: 30%

Additionally, individuals earning up to ₹7 lakh annually can receive a tax rebate, effectively exempting them from paying taxes if their taxable income is below this threshold. Surcharge and cess are applicable over and above these tax rates. Salaried employees and pensioners can also claim a standard deduction of ₹50,000 under the new regime.
